In [30]:
# LDA

In [31]:
from __future__ import print_function
import numpy as np
import pandas as pd
import pickle
import time

# Importing Gensim
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel

from IPython.display import SVG
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# import custom code
import os
import sys
pardir = os.path.abspath(os.path.join(os.getcwd(), '../'))
script_path = pardir + "/2_common_aux_script"
print('Importing process_string.py \nfrom ' + script_path + " ...\n")
sys.path.append(script_path)
from process_string import process_string
sys.path.remove(script_path)


print(sys.version)

Importing process_string.py 
from /Users/altay.amanbay/Desktop/new node booster/experiments/3a.1 - Nets train/5 train model - keras (unigrams)/2_common_aux_script ...

3.5.2 |Anaconda 4.2.0 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]


In [32]:
# Functions

def NGramGenerator_wordwise_interval(phrase, min_ngram, max_ngram):
    all_ngram_lists = []

    #printable_ = 'abcdefghijklmnopqrstuvwxyz0123456789 '
    #s_split = "".join((char if char in printable_ else "") for char in phrase).split()
    phrase_processed = process_string(phrase)
    s_split = phrase_processed.split()
    
    for n in range(max_ngram, min_ngram - 1, -1):
        n_gram = [s_split[i:i+n] for i in range(len(s_split)-n+1)]
        all_ngram_lists.extend(n_gram)
        
    all_ngrams = []
    for n_gram in all_ngram_lists:
        all_ngrams.extend([' '.join(n_gram)])
    
    return all_ngrams

def get_word2index(texts_ls_):
    word2index_ = {}

    c = 1
    for text_str in texts_ls_:
        text_tokens_ls = text_str.lower().split()
        for token in text_tokens_ls:
            if(token not in word2index_):
                word2index_[token] = c
                c = c + 1
                
    return word2index_

def train_df_preprocess(top_words_, texts_ls_, max_pad_length_):
    # texts_ls_: list of texts strings
    
    tok = Tokenizer(top_words_)
    tok.fit_on_texts(texts_ls_)

    words = []
    for iter in range(top_words):
        words += [key for key,value in tok.word_index.items() if value==iter+1]

    #Class for vectorizing texts, or/and turning texts into sequences 
    #(=list of word indexes, where the word of rank i in the dataset (starting at 1) has index i).
    texts_vec_ls = tok.texts_to_sequences(texts_ls_)#turns text to sequence, stating which word comes in what place
    texts_vec_mtx = sequence.pad_sequences(texts_vec_ls, maxlen=max_pad_length_)#pad sequence, essentially padding it with 0's at the end
    
    return texts_vec_mtx

def text_2_vec(text_str, word2index_):
    # text_str: text string
    
    text_tokens_ls = text_str.lower().split()
    
    text_vec = []
    for token in text_tokens_ls:
        if token in word2index_:
            text_vec.append(word2index_[token])
        else:
            text_vec.append(0)
            
    return text_vec

def train_df_preprocess_2(texts_ls_, word2index_, max_pad_length_):
    # texts_ls_: list of texts strings
    
    texts_vec_ls = []
    for text_ in texts_ls_:
        #print(text_)
        #print(type(text_))
        text_vec = text_2_vec(text_, word2index_)
        texts_vec_ls.append(text_vec)
    
    texts_vec_ary = sequence.pad_sequences(texts_vec_ls, maxlen=max_pad_length_)
    
    return texts_vec_ary

def texts_to_sequences_custom(texts_ls, word_index_, new_words_to_zero = False):
    texts_seq = []
    
    for text in texts_ls:
        #text_split = text.lower().split()
        text_split = NGramGenerator_wordwise_interval(text,1,1)
        seq = []
        for token in text_split:
            if(token in word_index_):
                seq.append(word_index_[token])
            elif(new_words_to_zero):
                seq.append(0)
                
        texts_seq.append(seq)
#         for k,v in word_index_.items():
#             if(v == 395):
#                 print(k,v)
    return texts_seq


def get_model_file_aux(model_file_aux_name):
    with open(model_file_aux_name, 'rb') as pickle_file:
        model_file_aux = pickle.load(pickle_file)
    return model_file_aux

def clone_tokens(text_str, cloning_factor = 1):
    token_clones_ls = []
    for token in text_str.split():
        token_clones_ls.extend([token]*cloning_factor)
    return ' '.join(token_clones_ls)

In [33]:
# Read sampled descriptionary

path = pardir+'/1_data/'
file_name = 'sampled_descriptionary_sample_size_5000.csv'
file_name = 'scorecards_for_fasttext.csv'
samples_df = pd.read_csv(path + file_name)

# Rename columns
samples_df.rename(columns={
                           #'description': 'description_mod1', 
                           '\ufeff"description"': 'description_mod1',
                           'category_id': 'category_id_mod1',
                           'category_path': 'category_full_path_mod1'}, inplace=True)

# Drop rows with NaN in any column
samples_df.dropna()

# Process description_mod1 strings by process_string function
samples_df['description_mod1'] = samples_df['description_mod1'].apply(lambda x: process_string(x))

# Drop rows where token count less than 1 in description_mod1 column
selected_indices = samples_df['description_mod1'].apply(lambda x: len(str(x).split()) > 1)
samples_df = samples_df[selected_indices]

# Drop duplicates
samples_df.drop_duplicates(subset=['description_mod1','category_full_path_mod1'], inplace = True, keep='first')
samples_df.drop_duplicates(subset=['description_mod1'], inplace = True, keep=False)
samples_df.reset_index(drop=True, inplace=True) 

# Clone tokens
samples_df['description_cloned'] = samples_df['description_mod1'].apply(lambda x: clone_tokens(x, cloning_factor=5))

# Drop 'screwdrivers' from descriptionary
#samples_df = samples_df.loc[samples_df.category_id_mod1 != 927,:]

# Drop index column
#samples_df.drop(labels=['index'], axis=1, inplace=True)

print('samples data shape:',samples_df.shape)
samples_df.head()

samples data shape: (587983, 4)


,description_mod1,category_id_mod1,category_full_path_mod1,description_cloned
0,aveeno baby wash shampoo lightly scented 8 ou...,206,Baby Products > Bathing & Skin Care > Soaps & ...,aveeno aveeno aveeno aveeno aveeno baby baby b...
1,earths best chlorine free diapers size 4 120 c...,213,Baby Products > Diapering > Disposable Diapers,earths earths earths earths earths best best b...
2,doa5lr costume catalog lr10 addon content,320,Electronics & Accessories > Video Games > Other,doa5lr doa5lr doa5lr doa5lr doa5lr costume cos...
3,lightweight oxford l in slim and white,152,Apparel & Accessories > Apparel > Tops & Tees ...,lightweight lightweight lightweight lightweigh...
4,now foods nutritional yeast flakes 10ounce,525,Health & Beauty > Vitamins & Dietary Supplements,now now now now now foods foods foods foods fo...


### Tweak unbiased data

In [34]:
## Check for class count

stats_sr = samples_df["category_full_path_mod1"].value_counts()
stats_sr[0:10]

Photos & Paper Products                                  50400
Books                                                    33416
Other                                                    29343
Health & Beauty > Makeup                                 24738
Online Services                                          22120
Health & Beauty > Vitamins & Dietary Supplements         20874
Jewelry & Watches > Earrings > Women                     18535
Apparel & Accessories > Apparel > Tops & Tees > Women    13440
Pet Supplies > Dogs & Cats > Dog Food & Treats           11791
Grocery & Gourmet Food > Restaurant & Takeout            10333
Name: category_full_path_mod1, dtype: int64

In [35]:
# Delete certain categories

samples_df = samples_df.loc[samples_df['category_full_path_mod1'].str.contains("Other")==False,:]
samples_df = samples_df.loc[samples_df.category_full_path_mod1 != "Books",:]
#samples_df = samples_df.loc[samples_df.category_full_path_mod1 != "Photos & Paper Products",:]

class_1 = 'Pet Supplies > Dogs & Cats > Dog Food & Treats'
class_1 = 'Photos & Paper Products'
class_2 = 'Apparel & Accessories > Apparel > Tops & Tees > Women'
class_2 = 'Apparel & Accessories > Accessories > Luggage, Backpacks & Laptop Bags'
classes_selected = class_1+'|'+class_2
#samples_df = samples_df.loc[samples_df['category_full_path_mod1'].str.contains(classes_selected)==True,:]
print('Pruned samples data shape:',samples_df.shape)

Pruned samples data shape: (484432, 4)


In [36]:
## Map class counts

samples_df['count'] = samples_df['category_full_path_mod1'].map(stats_sr)
samples_df.head()

,description_mod1,category_id_mod1,category_full_path_mod1,description_cloned,count
0,aveeno baby wash shampoo lightly scented 8 ou...,206,Baby Products > Bathing & Skin Care > Soaps & ...,aveeno aveeno aveeno aveeno aveeno baby baby b...,297
1,earths best chlorine free diapers size 4 120 c...,213,Baby Products > Diapering > Disposable Diapers,earths earths earths earths earths best best b...,2456
3,lightweight oxford l in slim and white,152,Apparel & Accessories > Apparel > Tops & Tees ...,lightweight lightweight lightweight lightweigh...,13440
4,now foods nutritional yeast flakes 10ounce,525,Health & Beauty > Vitamins & Dietary Supplements,now now now now now foods foods foods foods fo...,20874
5,navitas naturals organic goji berries 8 ounce ...,407,Grocery & Gourmet Food > Snack Foods > Dried F...,navitas navitas navitas navitas navitas natura...,182


In [37]:
## samples_df is unbiased by classes (category_full_path_mod1)
## fix by picking N samples from each class

#N=samples_df.category_full_path_mod1.value_counts(normalize=True).iloc[0] * samples_df.shape[0]
samples_df = samples_df.sample(frac=1).groupby('category_full_path_mod1', sort=False).head(1000)
print('samples data shape after picking max N samples from each class:',samples_df.shape)

samples data shape after picking max N samples from each class: (203321, 5)


### Concat sample_df into train_df

In [38]:
## Concat original train set and sampled descriptionary

#train_df = pd.concat([train_df, samples_df], axis=0)
train_df = samples_df
#train_df.reset_index(drop=True, inplace=True)

# description into chars
#train_df['description_mod1'] = train_df['description_mod1'].apply(lambda x: process_string(x))

# deduplicate
#train_df.drop_duplicates(subset=['description_mod1'], inplace = True, keep=False)
#print('train data shape (deduplicate):',train_df.shape)
    
# Encode target feature
le = LabelEncoder()
le.fit(train_df['category_full_path_mod1'])
train_df['target_le'] = le.transform(train_df['category_full_path_mod1'])


train_df.head(2)

,description_mod1,category_id_mod1,category_full_path_mod1,description_cloned,count,target_le
469009,marled vneck wedge tunic sweater,140,"Apparel & Accessories > Apparel > Sweaters, Sw...",marled marled marled marled marled vneck vneck...,2232,58
528968,magnet set of four 2x2 ads1 free,26,Photos & Paper Products,magnet magnet magnet magnet magnet set set set...,50400,603


In [47]:
# Create doc-term matrix

texts_ls = []
for idx, row in train_df.iterrows():
    texts_ls.append(row['description_mod1'].split())
    
texts_ls = [str(row['description_mod1']).split() for idx, row in train_df.iterrows()]
    
corp_dictionary = corpora.Dictionary(texts_ls)
doc_term_mtx = [corp_dictionary.doc2bow(text) for text in texts_ls]

In [65]:
i = 1
print(texts_ls[i])
doc_term_mtx[i]

['magnet', 'set', 'of', 'four', '2x2', 'ads1', 'free']


[(5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]

In [79]:
# Train LDA model
nb_classes = train_df['category_full_path_mod1'].unique()
print('Classes count:', len(nb_classes))

lda_model = LdaModel(corpus=doc_term_mtx
                     ,id2word=corp_dictionary
                     ,num_topics=200
                     ,update_every=1
                     ,chunksize=10000
                     ,passes=1)

Classes count: 861


KeyboardInterrupt: 

In [ ]:
N = 10
lda.printTopics(N)  # randomly picks N topics and shows words contribution

In [62]:
# simple experiment
documents = ["Apple is releasing a new product", 
             "Amazon sells many things is",
             "Microsoft announces Nokia acquisition"]

texts = [[word for word in document.lower().split()] for document in documents]
dictionary_ = corpora.Dictionary(texts)
corpus_ = [dictionary_.doc2bow(text) for text in texts]

print(texts)
corpus_



[['apple', 'is', 'releasing', 'a', 'new', 'product'], ['amazon', 'sells', 'many', 'things', 'is'], ['microsoft', 'announces', 'nokia', 'acquisition']]


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(0, 1), (6, 1), (7, 1), (8, 1), (9, 1)],
 [(10, 1), (11, 1), (12, 1), (13, 1)]]